In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import backend as K

from scipy.sparse import csr_matrix, vstack, hstack
from sklearn.metrics import roc_auc_score

In [3]:
train = pd.read_csv("train.csv").dropna()
test = pd.read_csv("test.csv").dropna()
y_train = np.array(train['Sentiment'])
y_test = np.array(test['Sentiment'])
X_train = train.drop(['Sentiment'], axis=1)
X_train = X_train.values
X_test = test.drop(['Sentiment'], axis=1).values
gen_features = ['word_count', 'unigram_count', 'unique_word_count', 'unique_unigram_count', 'stopword_count', 'mean_word_length', 'mean_unigram_length', 'char_count', 'punctuation_count', 'number_count']

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.legacy import Adam

model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1], kernel_regularizer=l2(0.0001)))
model.add(Dense(8, activation='relu', kernel_regularizer=l2(0.0001)))
model.add(Dense(1, activation='tanh'))  # Output layer


def custom_binary_crossentropy(y_true, y_pred):
    # Transform y_true to 0 and 1.
    y_true_transformed = K.cast((y_true + 1) / 2, dtype='float64')
    y_pred_transformed = K.cast((y_pred + 1) / 2, dtype='float64')

    # Compute binary crossentropy
    bce = K.binary_crossentropy(y_true_transformed, y_pred_transformed, from_logits=False)
    
    return K.mean(bce)

adam = Adam(learning_rate=0.0001)
def custom_accuracy(y_true, y_pred):
    # Since y_pred comes from tanh, values will be between -1 and 1.
    # We convert these to 1 and -1 for comparison.
    y_pred_labels = K.cast(K.greater(y_pred, 0), K.floatx()) * 2 - 1
    # Compare predicted labels to true labels
    correct_predictions = K.equal(y_true, y_pred_labels)
    # Calculate accuracy
    return K.mean(correct_predictions, axis=-1)

model.compile(optimizer=adam, loss=custom_binary_crossentropy, metrics=[custom_accuracy])
# Fit the model - assuming your data is ready and appropriately preprocessed
model.fit(X_train, y_train, epochs=30, batch_size=4, validation_split=0.1)

Epoch 1/30
1326/1326 [==============================] - 1s 541us/step - loss: 2.5028 - custom_accuracy: 0.5502 - val_loss: 0.0758 - val_custom_accuracy: 1.0000
Epoch 2/30
1326/1326 [==============================] - 1s 473us/step - loss: 0.7545 - custom_accuracy: 0.5967 - val_loss: 0.0412 - val_custom_accuracy: 1.0000
Epoch 3/30
1326/1326 [==============================] - 1s 472us/step - loss: 0.7396 - custom_accuracy: 0.5965 - val_loss: 0.1667 - val_custom_accuracy: 1.0000
Epoch 4/30
1326/1326 [==============================] - 1s 493us/step - loss: 0.7219 - custom_accuracy: 0.6050 - val_loss: 0.9624 - val_custom_accuracy: 0.2102
Epoch 5/30
1326/1326 [==============================] - 1s 476us/step - loss: 0.6986 - custom_accuracy: 0.6107 - val_loss: 0.0736 - val_custom_accuracy: 1.0000
Epoch 6/30
1326/1326 [==============================] - 1s 473us/step - loss: 0.6999 - custom_accuracy: 0.6160 - val_loss: 0.1177 - val_custom_accuracy: 1.0000
Epoch 7/30
1326/1326 [==================

In [16]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=8, max_value=64, step=8),
                    activation='relu', input_dim=X_train.shape[1],
                    kernel_regularizer=l2(hp.Float('l2', min_value=1e-4, max_value=1e-2, sampling='LOG'))))
    #model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units', min_value=8, max_value=32, step=8),
                    activation='relu',
                    kernel_regularizer=l2(hp.Float('l2', min_value=1e-4, max_value=1e-2, sampling='LOG'))))
    model.add(Dense(units=hp.Int('units', min_value=8, max_value=16, step=8),
                    activation='relu',
                    kernel_regularizer=l2(hp.Float('l2', min_value=1e-4, max_value=1e-2, sampling='LOG'))))
    # You can also add Dropout here if needed
    model.add(Dense(1, activation='tanh'))

    adam = Adam(learning_rate=hp.Float('learning_rate', min_value=0.00005, max_value=0.0003, step =0.00005))
    model.compile(optimizer=adam, loss=custom_binary_crossentropy, metrics=[custom_accuracy, 'val_accuracy'])

    return model

# Create a tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    hyperband_iterations=2)

# Perform hyperparameter tuning
tuner.search(X_train, y_train, epochs=50, validation_split=0.1, callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


Invalid metric: <function custom_accuracy at 0x29dc2ca40>


In [68]:

# You can now access the best hyperparameters as follows:
best_units = best_hps.get('units')
best_dropout = best_hps.get('dropout')
best_l2 = best_hps.get('l2')
best_learning_rate = best_hps.get('learning_rate')

# Now you can use these best hyperparameters to build the best model
best_model = tuner.hypermodel.build(best_hps)

# Summary of the best model
best_model.summary()

# You can now train the best model with the optimal hyperparameters
history = best_model.fit(X_train, y_train)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 208)               256880    
                                                                 
 dropout_2 (Dropout)         (None, 208)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 209       
                                                                 
Total params: 257089 (1004.25 KB)
Trainable params: 257089 (1004.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
185/185 [==============================] - 0s 1ms/step - loss: 3.8837 - accuracy: 0.2735


In [18]:
y_pred_test = model.predict(X_test)
roc_auc_test = roc_auc_score(y_test, y_pred_test)
print(f'AUC: {roc_auc_test}')

37/37 [==============================] - 0s 519us/step
AUC: 0.8030692183406394
